In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import ast

### Import first dataset

In [2]:
df_recipes2 = pd.read_parquet('../Data/recipes.parquet')
df_recipes2.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces..."
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,..."


In [3]:
df_recipes2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   RecipeId                    522517 non-null  float64            
 1   Name                        522517 non-null  object             
 2   AuthorId                    522517 non-null  int32              
 3   AuthorName                  522517 non-null  object             
 4   CookTime                    439972 non-null  object             
 5   PrepTime                    522517 non-null  object             
 6   TotalTime                   522517 non-null  object             
 7   DatePublished               522517 non-null  datetime64[ns, UTC]
 8   Description                 522512 non-null  object             
 9   Images                      522516 non-null  object             
 10  RecipeCategory              521766 non-null 

In [4]:
# Drop unnecessary columns
columns_to_drop = ['RecipeId', 'AuthorId', 'AuthorName', 'DatePublished', 'Description','RecipeYield']
df_recipes2 = df_recipes2.drop(columns=columns_to_drop)

In [5]:
# Convert arrays to lists
df_recipes2['RecipeInstructions'] = df_recipes2['RecipeInstructions'].apply(
    lambda x: list(x) if isinstance(x, (list, np.ndarray)) else []
)

### Import second dataset with units of measurement

In [6]:
df_2M = pd.read_csv('../Data/recipes_data.csv')

In [7]:
df_2M

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com
...,...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""marshmallows"", ...",www.foodnetwork.com
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""choice"", ""miracle whip"", ""eggs"", ""relish"", ""...",cookpad.com
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""soy sauce"", ""radish"", ""white sesame seeds"", ...",cookpad.com
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""egg"", ""sugar"", ""freshly groun...",cooking.nytimes.com


In [8]:
# Convert strings to lists
df_2M['directions'] = df_2M['directions'].fillna('[]').apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else []
)

### Merge the two datasets 

In [9]:
df_merged = pd.merge(df_recipes2, df_2M, left_on='Name', right_on='title', how='left')

In [10]:
df_merged.head()

,Name,CookTime,PrepTime,TotalTime,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,...,ProteinContent,RecipeServings,RecipeInstructions,title,ingredients,directions,link,source,NER,site
0,Low-Fat Berry Blue Frozen Dessert,PT24H,PT45M,PT24H45M,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[4, 1⁄4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...",4.5,...,3.2,4.0,"[Toss 2 cups berries with sugar., Let stand fo...",Low-Fat Berry Blue Frozen Dessert,"[""4 cups blueberries, fresh or frozen"", ""1/4 c...","[Toss 2 cups berries with sugar., Let stand fo...",www.food.com/recipe/low-fat-berry-blue-frozen-...,Gathered,"[""blueberries"", ""sugar"", ""vanilla yogurt"", ""le...",www.food.com
1,Biryani,PT25M,PT4H,PT4H25M,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",3.0,...,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,Biryani,"[""1/2 lb. meat or chicken, cut in pieces"", ""1/...",[Marinate meat or chicken in thick yogurt for ...,www.cookbooks.com/Recipe-Details.aspx?id=548876,Gathered,"[""meat"", ""ginger"", ""cinnamon"", ""cloves"", ""onio...",www.cookbooks.com
2,Biryani,PT25M,PT4H,PT4H25M,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",3.0,...,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,Biryani,"[""1 tablespoon saffron"", ""4 teaspoons milk, wa...",[Soak saffron in warm milk for 5 minutes and p...,www.food.com/recipe/biryani-39,Gathered,"[""tomatoes"", ""basmati rice"", ""coriander seed"",...",www.food.com
3,Biryani,PT25M,PT4H,PT4H25M,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",3.0,...,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,Biryani,"[""2 cups basmati rice"", ""34 kg chicken piece"",...","[Mix tomato puree, yogurt, ginger garlic paste...",www.food.com/recipe/biryani-484928,Recipes1M,"[""oil"", ""basmati rice"", ""cumin powder"", ""tomat...",www.food.com
4,Best Lemonade,PT5M,PT30M,PT35M,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,"[Low Protein, Low Cholesterol, Healthy, Summer...","[1 1⁄2, 1, None, 1 1⁄2, None, 3⁄4]","[sugar, lemons, rind of, lemon, zest of, fresh...",4.5,...,0.3,4.0,"[Into a 1 quart Jar with tight fitting lid, pu...",Best Lemonade,"[""1 1/2 c. sugar"", ""1 Tbsp. lemon peel, finely...","[In 1-quart jar with tight fitting lid, shake ...",www.cookbooks.com/Recipe-Details.aspx?id=293155,Gathered,"[""sugar"", ""lemon juice"", ""cold water"", ""very h...",www.cookbooks.com


#### Keep the identical recipes based on the instructions

In [11]:
filtered_df = df_merged[
    df_merged['RecipeInstructions'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None) ==
    df_merged['directions'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
].reset_index(drop=True)

In [12]:
filtered_df

,Name,CookTime,PrepTime,TotalTime,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,...,ProteinContent,RecipeServings,RecipeInstructions,title,ingredients,directions,link,source,NER,site
0,Low-Fat Berry Blue Frozen Dessert,PT24H,PT45M,PT24H45M,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[4, 1⁄4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...",4.5,...,3.2,4.0,"[Toss 2 cups berries with sugar., Let stand fo...",Low-Fat Berry Blue Frozen Dessert,"[""4 cups blueberries, fresh or frozen"", ""1/4 c...","[Toss 2 cups berries with sugar., Let stand fo...",www.food.com/recipe/low-fat-berry-blue-frozen-...,Gathered,"[""blueberries"", ""sugar"", ""vanilla yogurt"", ""le...",www.food.com
1,Biryani,PT25M,PT4H,PT4H25M,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",3.0,...,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,Biryani,"[""1 tablespoon saffron"", ""4 teaspoons milk, wa...",[Soak saffron in warm milk for 5 minutes and p...,www.food.com/recipe/biryani-39,Gathered,"[""tomatoes"", ""basmati rice"", ""coriander seed"",...",www.food.com
2,Cabbage Soup,PT30M,PT20M,PT50M,[https://img.sndimg.com/food/image/upload/w_55...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...","[46, 4, 1, 2, 1]","[plain tomato juice, cabbage, onion, carrots, ...",4.5,...,4.3,4.0,"[Mix everything together and bring to a boil.,...",Cabbage Soup,"[""46 ounces plain tomato juice"", ""4 cups cabba...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-42,Gathered,"[""cabbage"", ""onion"", ""celery"", ""carrots"", ""tom...",www.food.com
3,Cabbage Soup,PT30M,PT20M,PT50M,[https://img.sndimg.com/food/image/upload/w_55...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...","[46, 4, 1, 2, 1]","[plain tomato juice, cabbage, onion, carrots, ...",4.5,...,4.3,4.0,"[Mix everything together and bring to a boil.,...",Cabbage Soup,"[""1 head cabbage, chopped"", ""1 onion, diced"", ...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-204921,Recipes1M,"[""cabbage"", ""tomato paste"", ""pepper"", ""onion"",...",www.food.com
4,Buttermilk Pie With Gingersnap Crumb Crust,PT50M,PT30M,PT1H20M,[https://img.sndimg.com/food/image/upload/w_55...,Pie,"[Dessert, Healthy, Weeknight, Oven, < 4 Hours]","[3⁄4, 1, 1, 2, 3, 1⁄4, 1, 1⁄2, 1⁄2, 2]","[sugar, margarine, egg, flour, salt, buttermil...",4.0,...,4.2,8.0,"[Preheat oven to 350°F., Make pie crust, using...",Buttermilk Pie With Gingersnap Crumb Crust,"[""3/4 cup sugar"", ""1 tablespoon margarine"", ""1...","[Preheat oven to 350°F., Make pie crust, using...",www.food.com/recipe/buttermilk-pie-with-ginger...,Gathered,"[""buttermilk"", ""gingersnap crumbs"", ""egg"", ""su...",www.food.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318762,Skillet Chicken With Olives,PT10M,PT10M,PT20M,[],Chicken,"[Poultry, Meat, < 30 Mins, Easy]","[4, 1, 1⁄2, 1⁄4, 1, 1⁄2, 1⁄4, 1]","[boneless skinless chicken thighs, dried rosem...",NaN,...,13.6,4.0,"[Sprinkle chicken with rosemary, pepper and sa...",Skillet Chicken With Olives,"[""4 boneless skinless chicken thighs (about 1 ...","[Sprinkle chicken with rosemary, pepper and sa...",www.tasteofhome.com/recipes/skillet-chicken-wi...,Gathered,"[""pimiento"", ""pepper"", ""rosemary"", ""capers"", ""...",www.tasteofhome.com
318763,Bruschetta Steak,PT10M,PT15M,PT25M,[],Meat,"[< 30 Mins, Easy]","[3, 3, 3, 2, 1, 1, 3⁄4, 1, 1⁄4, 1]","[tomatoes, fresh basil, fresh parsley, olive o...",NaN,...,1.1,4.0,[Combine first six ingredients; stir in 1/4 te...,Bruschetta Steak,"[""3 medium tomatoes, chopped"", ""3 tablespoons ...",[Combine first six ingredients; stir in 1/4 te...,www.tasteofhome

In [13]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318767 entries, 0 to 318766
Data columns (total 29 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Name                        318767 non-null  object 
 1   CookTime                    264039 non-null  object 
 2   PrepTime                    318767 non-null  object 
 3   TotalTime                   318767 non-null  object 
 4   Images                      318767 non-null  object 
 5   RecipeCategory              318362 non-null  object 
 6   Keywords                    318767 non-null  object 
 7   RecipeIngredientQuantities  318767 non-null  object 
 8   RecipeIngredientParts       318767 non-null  object 
 9   AggregatedRating            174220 non-null  float64
 10  ReviewCount                 177831 non-null  float64
 11  Calories                    318767 non-null  float64
 12  FatContent                  318767 non-null  float64
 13  SaturatedFatCo

#### Drop duplicate columns

In [15]:
columns_to_drop = ['Name', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'RecipeInstructions']
df = filtered_df.drop(columns=columns_to_drop)

In [16]:
df

,CookTime,PrepTime,TotalTime,Images,RecipeCategory,Keywords,AggregatedRating,ReviewCount,Calories,FatContent,...,SugarContent,ProteinContent,RecipeServings,title,ingredients,directions,link,source,NER,site
0,PT24H,PT45M,PT24H45M,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...",4.5,4.0,170.9,2.5,...,30.2,3.2,4.0,Low-Fat Berry Blue Frozen Dessert,"[""4 cups blueberries, fresh or frozen"", ""1/4 c...","[Toss 2 cups berries with sugar., Let stand fo...",www.food.com/recipe/low-fat-berry-blue-frozen-...,Gathered,"[""blueberries"", ""sugar"", ""vanilla yogurt"", ""le...",www.food.com
1,PT25M,PT4H,PT4H25M,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...",3.0,1.0,1110.7,58.8,...,20.4,63.4,6.0,Biryani,"[""1 tablespoon saffron"", ""4 teaspoons milk, wa...",[Soak saffron in warm milk for 5 minutes and p...,www.food.com/recipe/biryani-39,Gathered,"[""tomatoes"", ""basmati rice"", ""coriander seed"",...",www.food.com
2,PT30M,PT20M,PT50M,[https://img.sndimg.com/food/image/upload/w_55...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...",4.5,11.0,103.6,0.4,...,17.7,4.3,4.0,Cabbage Soup,"[""46 ounces plain tomato juice"", ""4 cups cabba...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-42,Gathered,"[""cabbage"", ""onion"", ""celery"", ""carrots"", ""tom...",www.food.com
3,PT30M,PT20M,PT50M,[https://img.sndimg.com/food/image/upload/w_55...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...",4.5,11.0,103.6,0.4,...,17.7,4.3,4.0,Cabbage Soup,"[""1 head cabbage, chopped"", ""1 onion, diced"", ...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-204921,Recipes1M,"[""cabbage"", ""tomato paste"", ""pepper"", ""onion"",...",www.food.com
4,PT50M,PT30M,PT1H20M,[https://img.sndimg.com/food/image/upload/w_55...,Pie,"[Dessert, Healthy, Weeknight, Oven, < 4 Hours]",4.0,3.0,228.0,7.1,...,24.7,4.2,8.0,Buttermilk Pie With Gingersnap Crumb Crust,"[""3/4 cup sugar"", ""1 tablespoon margarine"", ""1...","[Preheat oven to 350°F., Make pie crust, using...",www.food.com/recipe/buttermilk-pie-with-ginger...,Gathered,"[""buttermilk"", ""gingersnap crumbs"", ""egg"", ""su...",www.food.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318762,PT10M,PT10M,PT20M,[],Chicken,"[Poultry, Meat, < 30 Mins, Easy]",NaN,NaN,125.6,6.1,...,0.1,13.6,4.0,Skillet Chicken With Olives,"[""4 boneless skinless chicken thighs (about 1 ...","[Sprinkle chicken with rosemary, pepper and sa...",www.tasteofhome.com/recipes/skillet-chicken-wi...,Gathered,"[""pimiento"", ""pepper"", ""rosemary"", ""capers"", ""...",www.tasteofhome.com
318763,PT10M,PT15M,PT25M,[],Meat,"[< 30 Mins, Easy]",NaN,NaN,80.4,7.0,...,2.5,1.1,4.0,Bruschetta Steak,"[""3 medium tomatoes, chopped"", ""3 tablespoons ...",[Combine first six ingredients; stir in 1/4 te...,www.tasteofhome.com/recipes/bruschetta-steak/,Gathered,"[""tomatoes"", ""pepper"", ""fresh oregano"", ""parme...",www.tasteofhome.com
318764,PT15M,PT15M,PT30M,[],Vegetable,"[< 30 Mins, Easy]",NaN,NaN,273.6,3.1,...,4.1,17.5,6.0,Tomato-Garlic Lentil Bowls,"[""1 tablespoon olive oil"", ""2 medium onions, c...","[In a large saucepan, heat oil over medium-hig...",www.tasteofhome.com/recipes/tomato-garlic-lent...,Gathered,"[""tomatoes"", ""pepper"", ""lemon juice"", ""tomato ...",www.tasteofhome.com
318765,PT10M,PT10M,PT20M,[],Very Low Carbs,"[< 30 Mins, Easy]",NaN,NaN,180.1,6.1,...,1.6,27.7,6.0,Mediterranean Tilapia,"[""6 tilapia fillets (6 ounces each)"", ""1 cup c...",[Preheat oven to 400°. Place fillets in a 15x1...,www.tasteofhome.com/recipes/mediterranean-tila...,Gathered,"[""tomatoes"", ""water"", ""feta cheese"", ""olives"",...",www.tasteofhome.com


## Cleaning

### Columns CookTime, PrepTime and TotalTime

In [22]:
def duration_to_minutes(duration: str) -> float:
    """
    Function to convert ISO 8601 durations to total minutes

    Args:
        - duration (str): duration in ISO 8601 format (ex: 'PT1H30M')
    """
    if pd.isna(duration): 
        return np.nan

    hours = re.search(r'(\d+)H', duration)
    minutes = re.search(r'(\d+)M', duration)
    
    total_minutes = 0
    if hours:
        total_minutes += int(hours.group(1)) * 60
    if minutes:
        total_minutes += int(minutes.group(1))
    
    return total_minutes

In [19]:
def duration_to_readable_format(duration: str) -> str:
    """
    Function to convert ISO 8601 durations to a more readable format
    
    Args:
        - duration (str): duration in ISO 8601 format (ex: 'PT1H30M')

    Example output: "1 hour 30 minutes"
    """
    if pd.isna(duration): 
        return np.nan

    hours = re.search(r'(\d+)H', duration)
    minutes = re.search(r'(\d+)M', duration)
    
    result = []
    if hours:
        result.append(f"{int(hours.group(1))} hour{'s' if int(hours.group(1)) > 1 else ''}")
    if minutes:
        result.append(f"{int(minutes.group(1))} minute{'s' if int(minutes.group(1)) > 1 else ''}")
    
    return ' '.join(result)

In [23]:
columns = ['CookTime', 'PrepTime', 'TotalTime']
for col in columns:
    df[f'{col}_minutes'] = df[col].apply(duration_to_minutes)

In [ ]:
# Fill CookTime nan values with 0
df['CookTime_minutes'] = df['CookTime_minutes'].fillna(0)

### Column Images : keep the first link only

In [24]:
def extract_first_element(x):
    if isinstance(x, np.ndarray): 
        return x[0] if x.size > 0 else None  
    elif pd.notna(x):  
        return x  
    else:
        return None

In [25]:
df['Images'] = df['Images'].apply(extract_first_element)

In [27]:
df

,CookTime,PrepTime,TotalTime,Images,RecipeCategory,Keywords,AggregatedRating,ReviewCount,Calories,FatContent,...,title,ingredients,directions,link,source,NER,site,CookTime_minutes,PrepTime_minutes,TotalTime_minutes
0,PT24H,PT45M,PT24H45M,https://img.sndimg.com/food/image/upload/w_555...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...",4.5,4.0,170.9,2.5,...,Low-Fat Berry Blue Frozen Dessert,"[""4 cups blueberries, fresh or frozen"", ""1/4 c...","[Toss 2 cups berries with sugar., Let stand fo...",www.food.com/recipe/low-fat-berry-blue-frozen-...,Gathered,"[""blueberries"", ""sugar"", ""vanilla yogurt"", ""le...",www.food.com,1440.0,45,1485
1,PT25M,PT4H,PT4H25M,https://img.sndimg.com/food/image/upload/w_555...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...",3.0,1.0,1110.7,58.8,...,Biryani,"[""1 tablespoon saffron"", ""4 teaspoons milk, wa...",[Soak saffron in warm milk for 5 minutes and p...,www.food.com/recipe/biryani-39,Gathered,"[""tomatoes"", ""basmati rice"", ""coriander seed"",...",www.food.com,25.0,240,265
2,PT30M,PT20M,PT50M,https://img.sndimg.com/food/image/upload/w_555...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...",4.5,11.0,103.6,0.4,...,Cabbage Soup,"[""46 ounces plain tomato juice"", ""4 cups cabba...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-42,Gathered,"[""cabbage"", ""onion"", ""celery"", ""carrots"", ""tom...",www.food.com,30.0,20,50
3,PT30M,PT20M,PT50M,https://img.sndimg.com/food/image/upload/w_555...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...",4.5,11.0,103.6,0.4,...,Cabbage Soup,"[""1 head cabbage, chopped"", ""1 onion, diced"", ...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-204921,Recipes1M,"[""cabbage"", ""tomato paste"", ""pepper"", ""onion"",...",www.food.com,30.0,20,50
4,PT50M,PT30M,PT1H20M,https://img.sndimg.com/food/image/upload/w_555...,Pie,"[Dessert, Healthy, Weeknight, Oven, < 4 Hours]",4.0,3.0,228.0,7.1,...,Buttermilk Pie With Gingersnap Crumb Crust,"[""3/4 cup sugar"", ""1 tablespoon margarine"", ""1...","[Preheat oven to 350°F., Make pie crust, using...",www.food.com/recipe/buttermilk-pie-with-ginger...,Gathered,"[""buttermilk"", ""gingersnap crumbs"", ""egg"", ""su...",www.food.com,50.0,30,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318762,PT10M,PT10M,PT20M,None,Chicken,"[Poultry, Meat, < 30 Mins, Easy]",NaN,NaN,125.6,6.1,...,Skillet Chicken With Olives,"[""4 boneless skinless chicken thighs (about 1 ...","[Sprinkle chicken with rosemary, pepper and sa...",www.tasteofhome.com/recipes/skillet-chicken-wi...,Gathered,"[""pimiento"", ""pepper"", ""rosemary"", ""capers"", ""...",www.tasteofhome.com,10.0,10,20
318763,PT10M,PT15M,PT25M,None,Meat,"[< 30 Mins, Easy]",NaN,NaN,80.4,7.0,...,Bruschetta Steak,"[""3 medium tomatoes, chopped"", ""3 tablespoons ...",[Combine first six ingredients; stir in 1/4 te...,www.tasteofhome.com/recipes/bruschetta-steak/,Gathered,"[""tomatoes"", ""pepper"", ""fresh oregano"", ""parme...",www.tasteofhome.com,10.0,15,25
318764,PT15M,PT15M,PT30M,None,Vegetable,"[< 30 Mins, Easy]",NaN,NaN,273.6,3.1,...,Tomato-Garlic Lentil Bowls,"[""1 tablespoon olive oil"", ""2 medium onions, c...","[In a large saucepan, heat oil over medium-hig...",www.tasteofhome.com/recipes/tomato-garlic-lent...,Gathered,"[""tomatoes"", ""pepper"", ""lemon juice"", ""tomato ...",www.tasteofhome.com,15.0,15,30
318765,PT10M,PT10M,PT20M,None,Very Low Carbs,"[< 30 Mins, Easy]",NaN,NaN,180.1,6.1,...,Mediterranean Tilapia,"[""6 tilapia fillets (6 ounces each)"", ""1 cup c...",[Preheat oven to 400°. Place fillets in a 15x1...,www.tasteofhome.com/recipes/mediterranean-tila...,Gathered,"[""tomatoes"", ""water"", ""feta cheese"", ""olives"",...",www.tasteofhome.com,10.0,10,20


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318767 entries, 0 to 318766
Data columns (total 28 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CookTime             264039 non-null  object 
 1   PrepTime             318767 non-null  object 
 2   TotalTime            318767 non-null  object 
 3   Images               100491 non-null  object 
 4   RecipeCategory       318362 non-null  object 
 5   Keywords             318767 non-null  object 
 6   AggregatedRating     174220 non-null  float64
 7   ReviewCount          177831 non-null  float64
 8   Calories             318767 non-null  float64
 9   FatContent           318767 non-null  float64
 10  SaturatedFatContent  318767 non-null  float64
 11  CholesterolContent   318767 non-null  float64
 12  SodiumContent        318767 non-null  float64
 13  CarbohydrateContent  318767 non-null  float64
 14  FiberContent         318767 non-null  float64
 15  SugarContent     

### Column RecipeCategory

In [39]:
df['RecipeCategory']

0         Frozen Desserts
1          Chicken Breast
2               Vegetable
3               Vegetable
4                     Pie
               ...       
318762            Chicken
318763               Meat
318764          Vegetable
318765     Very Low Carbs
318766               Pork
Name: RecipeCategory, Length: 318767, dtype: object

In [36]:
df['RecipeCategory'].nunique()

297